In [5]:
import os
import sys
import pickle
import warnings
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from Soroosh_utilities import *

warnings.filterwarnings("ignore")




In [6]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# dtw, path = fastdtw(x, y, dist=euclidean)



clone to the following repository 

https://github.com/vanderschaarlab/mlforhealthlabpub.git

and set the path accordingly

In [7]:
path = "/home/soroosh/TimeGAN/"
sys.path.append(path)



In [8]:
## Necessary packages
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


# 1. TimeGAN model
from timegan import timegan
# 2. Data loading
from data_loading import real_data_loading, sine_data_generation
# 3. Metrics
from metrics.discriminative_metrics import discriminative_score_metrics
from metrics.predictive_metrics import predictive_score_metrics
from metrics.visualization_metrics import visualization




ATTENTION PERMUTATION IS: False
ATTENTION PERMUTATION IS: False
ATTENTION PERMUTATION IS: False


In [9]:
def prepare_data_for_tgan(x, seq_length):
    
    dataX = []
    
    # Cut data by sequence length
    for i in range(0, len(x) - seq_length):
        _x = x[i:i + seq_length]
        dataX.append(_x)
    
#     # Mix Data (to make it similar to i.i.d)
#     idx = np.random.permutation(len(dataX))
    
#     outputX = []
#     for i in range(len(dataX)):
#         outputX.append(dataX[idx[i]])
#     return outputX
    return dataX
    



### Dataset

In [10]:
# n_time_series = 10
# x_reduced = [str(i) for i in range(n_time_series)]
# x_reduced

# x_random = {}
# for i in range(n_time_series):
#     x_random[x_reduced[i]] = np.random.uniform(low=0, 
#                                                 high=1,
#                                                 size=500)
    
# x_random_df = pd.DataFrame(data=x_random)


# pld_complete_range = x_random_df
# pld_complete_range



In [11]:
pld_complete_range = pd.read_csv("/home/soroosh/Desktop/SearchOX/data/pld_complete_range.csv",
                                 index_col=False)


# pld_complete_range.head()



In [12]:
pos_idx = pld_complete_range.columns.get_loc("pos")
ic_idx = pld_complete_range.columns.get_loc("Ic")
ic_norm_idx = pld_complete_range.columns.get_loc("Ic_norm")

pos_idx, ic_idx, ic_norm_idx



(370, 377, 378)

In [13]:
# for i in range(379):
#     print(pld_complete_range.columns[i], "# unique values of :"
#           ,len(set(pld_complete_range.iloc[:, i].values)))

In [14]:

features_name = pld_complete_range.columns.to_list()


In [15]:
without_stats_features = ['Speed', 'X FWHM', 'Y FWHM', 'R FWHM',
                          'Coolness', 'Coolness_neg', 'Ic', 'Ic_norm']  # 'pos',



In [16]:


features_to_keep_names = []

for feature in features_name:
    if "mean" in feature:
        features_to_keep_names.append(feature)
        

In [17]:
for f in without_stats_features:
    features_to_keep_names.append(f)
    
    

In [18]:
features_to_keep_indices = []

for feature in features_to_keep_names:
    features_to_keep_indices.append(features_name.index(feature))
    
    

In [19]:
len(features_to_keep_indices)



105

In [20]:
pld_complete_range_non_stats = pld_complete_range.loc[:, without_stats_features]



In [21]:
# pld_complete_range_non_stats


In [22]:
with open ("/home/soroosh/Desktop/SearchOX/data/controllable_features.pickle", "rb") as fp:
    controllable_features = pickle.load(fp)
    

with open ("/home/soroosh/Desktop/SearchOX/data/physics_related_features.pickle", "rb") as fp:
    physics_related_features = pickle.load(fp)
    

In [ ]:
pld_complete_range_reduced_phy = pld_complete_range.loc[:, physics_related_features]
# pld_complete_range_reduced_phy


In [23]:
def save_results(dataX, dataX_hat, seq_len, parameters, n_time_series, name):

    print(seq_len, "\n", 
         parameters
         )
    
    # save the results flatted along the windows_len
    gen_data_arr = np.zeros([len(dataX_hat)*seq_len, n_time_series])
    org_data_arr = np.zeros([len(dataX_hat)*seq_len, n_time_series])


    interval = 0
    for i in range(len(dataX_hat)):
        for j in range(seq_len):
            gen_data_arr[interval, :] = dataX_hat[i][j, :]
            org_data_arr[interval, :] = dataX[i][j, :]
            interval += 1    

    interval = 0
    for idx in range(len(dataX_hat)):
        for j in range(seq_len): 
            if len(set(gen_data_arr[interval, :] == dataX_hat[idx][j, :])) != 1:
                print(idx)
            if len(set(org_data_arr[interval, :] == dataX[idx][j, :])) != 1:
                print(idx)

            interval += 1
    
     # flatted data for DTW
    np.savetxt("gan_data/gen_data_gan-"+\
               str(seq_len) + "-" + str(parameters["num_layers"])+\
               "-" + str(parameters["iterations"])+\
               name, 
          gen_data_arr)
    
        
    np.savetxt("gan_data/org_data_gan-"+\
               str(seq_len) + "-" + str(parameters["num_layers"])+\
               "-" + str(parameters["iterations"])+\
               name, 
              org_data_arr)
    
    with open ("gan_data/dataX" +\
               str(seq_len) + "-" +\
               str(parameters["num_layers"])+\
               "-" + str(parameters["iterations"])+\
               name, "wb") as fp:
        pickle.dump(dataX, fp)
        
        
    with open ("gan_data/dataX_hat" +\
               str(seq_len) + "-" +\
               str(parameters["num_layers"])+\
               "-" + str(parameters["iterations"])+\
               name, "wb") as fp:
        pickle.dump(dataX_hat, fp)
        
    
    return None



In [24]:
def eval_scores(dataX, dataX_hat):
    
    # Output Initialization
    Discriminative_Score = list()
    Predictive_Score = list()

    Acc = list()
    Sub_Iteration = 2
    for tt in range(Sub_Iteration):
        Temp_Disc = discriminative_score_metrics(dataX, dataX_hat)
        Acc.append(Temp_Disc)

    Discriminative_Score.append(np.mean(Acc))

    # 2. Predictive Performance
    MAE_All = list()
    for tt in range(Sub_Iteration):
        MAE_All.append(predictive_score_metrics (dataX, dataX_hat))

    Predictive_Score.append(np.mean(MAE_All))    
    
    # Print Results
    print('Discriminative Score - Mean: ' + str(np.round(np.mean(Discriminative_Score), 4)) + ', Std: ' + str(np.round(np.std(Discriminative_Score),4)))
    print('Predictive Score - Mean: ' + str(np.round(np.mean(Predictive_Score),4)) + ', Std: ' + str(np.round(np.std(Predictive_Score),4)))
    
    return Discriminative_Score, Predictive_Score


    

In [25]:
def load_results(seq_len, parameters, name):
    
    print("Loading the saved results and data")
    
    with open ("gan_data/dataX" +\
               str(seq_len) + "-" +\
               str(parameters["num_layers"])+\
               "-" + str(parameters["iterations"])+\
               name, "rb") as fp:
        dataX=pickle.load(fp)
    
    
    with open ("gan_data/dataX_hat" +\
               str(seq_len) + "-" +\
               str(parameters["num_layers"])+\
               "-" + str(parameters["iterations"])+\
               name, "rb") as fp:
        dataX_hat=pickle.load(fp)
        
    
    # flatted and saved data for DTW
    org_data_gan = np.loadtxt("gan_data/org_data_gan-"+\
                              str(seq_len) + "-" +\
                              str(parameters["num_layers"])+\
                              "-" + str(parameters["iterations"])+\
                              name
                             )
    
    
    gen_data_gan = np.loadtxt("gan_data/gen_data_gan-"+\
                              str(seq_len) + "-" +\
                              str(parameters["num_layers"])+\
                              "-" + str(parameters["iterations"])+\
                              name
                             )

    
    return dataX, dataX_hat, org_data_gan, gen_data_gan
    

### TSGAN (Time-Series)

Paper: https://papers.nips.cc/paper/2019/file/c9efe5f26cd17ba6216bbe2a7d26d490-Paper.pdf

Source code: https://github.com/vanderschaarlab/mlforhealthlabpub/tree/main/alg/timegan



###### Define Model hyperparameters
- Networks: Generator; Discriminator; Embedder; Recovery Network

TimeGAN is a Generative model based on RNN networks. In this package the implemented version follows a very simple architecture that is shared by the four elements of the GAN.
Similarly to other parameters, the architectures of each element should be optimized and tailored to the data.

In [26]:
# project = "SuperOX"
# data_type = "reduced statistical features-dtw"  # "all statistical features"



#     run = init_a_wandb(name= "GMM: " + str(hyper_param),
#                        project=project,
#                        notes="Generating independent features - GMM " + data_type, 
#                        group= "GMM + CGAN - " + data_type  # algorithm
#                       )
    


#     run = wandb_metrics(run=run, y_trues=x_i_test, y_preds=x_i_hat)
#     run = wandb_plot_predictions(run=run, algorithm= "gmm: " + str(hyper_param), 
#                                  y_trues=y_test, y_preds=y_i_hat.values.squeeze())
    
#     run = save_model(run=run, 
#                      model=gmm, 
#                      name="gmm", 
#                      experiment_name="gmm 4 x_i -" + data_type + "-" + str(hyper_param)
#                     )  

In [27]:
# np.random.seed(42)
# n_samples = pld_complete_range_reduced.shape[0]
# n_train_samples = int(.7*n_samples)
# all_indices = np.arange(n_samples).tolist()
# train_indices = np.random.choice(all_indices, n_train_samples, replace=False).tolist()
# test_indices = list(set(all_indices) - set(train_indices))



## Train with different parameters:



In [28]:
sequence_lens = [3, 5, 10, 50] # 1 >> generate error in discriminative/predictive score
number_layers = [3, 5]  

iterations= 10000 
name= "non-stats-no-perem"  # physics-perm, # physics-no-perm, "non-stats-perem", 


name



'non-stats-no-perem'

In [29]:
def train_and_eval(data_to_use, 
                   sequence_lens,
                   number_layers, 
                   name):
    
    for seq_len in sequence_lens:
        for num_layers in number_layers:
            print("seq_len:", seq_len, "num_layers:", num_layers)
            data_to_train = data_to_use.values
            n_samples = data_to_train.shape[0]
            n_time_series = data_to_train.shape[1]

            dataX = prepare_data_for_tgan(x=data_to_train, seq_length=seq_len)

            print("dataX[0].shape:", dataX[0].shape)

            #%% Newtork Parameters
            parameters = dict()

            parameters['hidden_dim'] = len(dataX[0][0,:]) * 4
            parameters["num_layers"] = num_layers
            parameters['iterations'] = iterations
            
            parameters['batch_size'] = 128
            parameters['module'] = 'gru'   # Other options: 'lstm' or 'lstmLN'

            parameters['z_dim'] = len(dataX[0][0,:]) 
            
            
            
            print('Parameters are ' + str(parameters))
            print(" ")

            dataX_hat = timegan(dataX, parameters)   

            print('Finish Synthetic Data Generation')

            save_results(dataX=dataX, dataX_hat=dataX_hat, 
                         seq_len=seq_len,
                         parameters=parameters, 
                         n_time_series=n_time_series,
                         name=name
                        )
        
    return None



In [ ]:
train_and_eval(data_to_use=pld_complete_range_non_stats, 
               sequence_lens=sequence_lens,
               number_layers=number_layers,
               name=name)



seq_len: 3 num_layers: 3
dataX[0].shape: (3, 8)
Parameters are {'hidden_dim': 32, 'num_layers': 3, 'iterations': 10000, 'batch_size': 128, 'module': 'gru', 'z_dim': 8}
 
Start Embedding Network Training
step: 0/10000, e_loss: 0.2663
step: 1000/10000, e_loss: 0.0184
step: 2000/10000, e_loss: 0.0184
step: 3000/10000, e_loss: 0.0184
step: 4000/10000, e_loss: 0.0134
step: 5000/10000, e_loss: 0.0093
step: 6000/10000, e_loss: 0.0071
step: 7000/10000, e_loss: 0.0052
step: 8000/10000, e_loss: 0.0046
step: 9000/10000, e_loss: 0.0039
Finish Embedding Network Training
Start Training with Supervised Loss Only
step: 0/10000, s_loss: 0.2243
step: 1000/10000, s_loss: 0.0916
step: 2000/10000, s_loss: 0.073
step: 3000/10000, s_loss: 0.068
step: 4000/10000, s_loss: 0.0639
step: 5000/10000, s_loss: 0.0596
step: 6000/10000, s_loss: 0.0553
step: 7000/10000, s_loss: 0.0525
step: 8000/10000, s_loss: 0.0493
step: 9000/10000, s_loss: 0.0471
Finish Training with Supervised Loss Only
Start Joint Training
step: 0

In [ ]:
# train_and_eval(data_to_use=pld_complete_range_reduced_phy, 
#                sequence_lens=sequence_lens,
#                number_layers=number_layers,
#                name="physics")



## Evaluate the obtained (and saved) results

In [ ]:
print("finish")


In [ ]:

def eval_the_stored_results(name):
    # name=""  # "" for toy data, "physics", "controlable"
    
    sequence_lens = [3, 5, 10, 50] 
    number_layers = [3, 5,] 
    
    discriminative_score = {}
    predictive_score = {}
    dtw_score = {}


    for seq_len in sequence_lens:
        for num_layers in number_layers:
            print("seq_len:", seq_len, "num_layers:", num_layers)

            # data_to_use = pld_complete_range_reduced.values
            # n_samples = data_to_use.shape[0]
            # n_time_series = data_to_use.shape[1]


            #%% Newtork Parameters
            parameters = dict()

            # parameters['hidden_dim'] = len(dataX[0][0,:]) * 4
            parameters['num_layers'] = num_layers
            parameters['iterations'] = iterations
            parameters['batch_size'] = 128
            parameters['module_name'] = 'gru'   # Other options: 'lstm' or 'lstmLN'
            # parameters['z_dim'] = len(dataX[0][0,:]) 

            print('Parameters are ' + str(parameters))
            print(" ")

            d, dh, org, gen = load_results(seq_len=seq_len, 
                                           parameters=parameters, 
                                           name=name)
            
            print("dataX[0].shape:", d[0].shape, len(d), "\n", 
                 "dataX_hat[0].shape:", dh[0].shape, len(dh)

                 )
            disc_scr, pred_scr = eval_scores(dataX=d, dataX_hat=dh)


            dtw_tgan, _ = fastdtw(x=org, 
                                  y=gen,
                                  dist=None)  # None then abs(x[i] - y[j]) will be used.

            discriminative_score[str(seq_len)+"-"+str(num_layers)] = disc_scr

            predictive_score[str(seq_len)+"-"+str(num_layers)] = pred_scr

            dtw_score[str(seq_len)+"-"+str(num_layers)] = dtw_tgan



    with open("gan_data/discriminative_score" + name + ".pickle", "wb") as fp:
        pickle.dump(discriminative_score, fp)


    with open("gan_data/predictive_score" + name + ".pickle", "wb") as fp:
        pickle.dump(predictive_score, fp)


    with open("gan_data/dtw_score" + name + ".pickle", "wb") as fp:
        pickle.dump(dtw_score, fp)
    
    return discriminative_score, predictive_score, dtw_score


In [ ]:
discriminative_score_toy, predictive_score_toy, dtw_score_toy = eval_the_stored_results(name=name)



In [ ]:
discriminative_score_phy, predictive_score_phy, dtw_score_phy = eval_the_stored_results(name="physics")




In [ ]:
# with open("gan_data/discriminative_score.pickle", "rb") as fp:
#     ds = pickle.load(fp)
    

# with open("gan_data/predictive_score.pickle", "rb") as fp:
#     ps = pickle.load(fp)
    
    
# with open("gan_data/dtw_score.pickle", "rb") as fp:
#     dtw = pickle.load(fp)

In [ ]:
# dataX_org_1half = prepare_data_for_tgan(x=data_to_train[:14000, ], seq_length=3)
# dataX_org_2half = prepare_data_for_tgan(x=data_to_train[14000:, ], seq_length=3)

# disc_scr_org_half, pred_scr_org_half = eval_scores(dataX=dataX_org_1half, dataX_hat=dataX_org_2half)



##  Discriminative Score:

For a quantitative measure of similarity, we train a post-hoc time-series classification model (by optimizing a 2-layer LSTM) to distinguish between sequences from the original and generated datasets. First, each original sequence is labeled real, and each generated sequence is labeled not real. Then, an off-the-shelf (RNN) classifier is trained to distinguish between the two classes as a standard supervised task. We then report the classification error on the held-out test set, which gives a quantitative assessment of (2).

-  **(Lower the better)**

##  Predictive Score:

In order to be useful, the sampled data should inherit the predictive characteris- tics of the original. In particular, we expect TimeGAN to excel in capturing conditional distributions over time. Therefore, using the synthetic dataset, we train a post-hoc sequence-prediction model (by optimizing a 2-layer LSTM) to predict next-step temporal vectors over each input sequence. Then, we evaluate the trained model on the original dataset. Performance is measured in terms of the mean absolute error (MAE); for event-based data, the MAE is computed as |1− estimated probability that the event occurred|. This gives a quantitative assessment of (3).

-  **Lower the better**



In [ ]:
print("ON ORIGINALN DATA"
         "\n", 
         "    disc. score: %.4f" % disc_scr_org[0],
          "\t pred. score: %.4f" %  pred_scr_ord[0],
#           "\t dtw: %4f" % dtw_score[k], 
         )

for k, v in discriminative_score.items():
    sl = k.split('-')[0]
    nl = k.split('-')[1]
    print("seq_len:", sl, "\t num_layers:", nl, 
         "\n", 
         "    disc. score: %.4f" % v[0],
          "\t pred. score: %.4f" %  predictive_score[k][0],
#           "\t dtw: %4f" % dtw_score[k], 
         )
    
    

- All of the obtained results are  close to the case when we apply the same framework on the original data.

- Thus we can conclude that the generated data's quality is similar the original data

- Please not that, in the experiment above our goal is not to obtain a low pred/disc score, but the goal is to obtain similar results when we usued the original data with the same prediction/discriminator algorithm.

In [ ]:
dtw_org, _ = fastdtw(x=data_to_train, y=data_to_train, dist=None)  # None then abs(x[i] - y[j]) will be used.
dtw_org

dtw_org_half, _ = fastdtw(x=data_to_train[:5500, :],
                          y=data_to_train[5500:, :], 
                          dist=None)  # None then abs(x[i] - y[j]) will be used.

dtw_org_half


# dtw, path = fastdtw(x, y, dist=euclidean)
rnd_idx = np.random.choice(np.arange(0, n_samples), size=n_samples, replace=False)
data_to_train_shuf = data_to_train[rnd_idx, :]
dtw_shuf, _ = fastdtw(x=data_to_train, y=data_to_train_shuf, dist=None)  # None then abs(x[i] - y[j]) will be used.

dtw_shuf

x_random = {}
for i in range(data_to_train.shape[1]):
    x_random[i] = np.random.uniform(low=-1,  # x_reduced.iloc[:, i].min() 
                                    high=+1,  # x_reduced.iloc[:, i].max()
                                    size=n_samples)
    
x_random_df = pd.DataFrame(data=x_random)


dtw_rnd, _ = fastdtw(x=data_to_train, y=x_random_df, dist=None)  # None then abs(x[i] - y[j]) will be used.

print(" DTW of Original Data    : %.3f" % dtw_org, "\n", 
      "DTW of Shuffled Data    : %.3f" % dtw_shuf, "\n",  
      "DTW of half Data        : %.3f" % dtw_org_half, "\n"
      " DTW of Random Data      : %.3f" % dtw_rnd, "\n", 
     )


for k, v in discriminative_score.items():
    sl = k.split('-')[0]
    nl = k.split('-')[1]
    print("seq_len:", sl, "\t num_layers:", nl, 
          "\t DTW: %.3f" % dtw_score[k], 
         )
    
    



- The minimum DTW obtained when seq_len=3, num_layers=5
- Comparing the DTW of the three first items with the Random Data, we can conclude that, even regarding this metrics also the quality is acceptable.


# Visualization

In [ ]:
for seq_len in sequence_lens:
    for num_layers in number_layers:
        
        data_to_train = pld_complete_range_reduced.values
        n_samples = data_to_train.shape[0]
        n_time_series = data_to_train.shape[1]
        

        #%% Newtork Parameters
        parameters = dict()

        # parameters['hidden_dim'] = len(dataX[0][0,:]) * 4
        parameters['num_layers'] = num_layers
        parameters['iterations'] = iterations
        parameters['batch_size'] = 128
        parameters['module_name'] = 'gru'   # Other options: 'lstm' or 'lstmLN'
        # parameters['z_dim'] = len(dataX[0][0,:]) 

#         print('Parameters are ' + str(parameters))
#         print(" ")
        
        d, dh, org, gen = load_results(seq_len=seq_len,
                                       parameters=parameters, 
                                       name=name)
        
        print("seq_len:", seq_len, "\t num_layers:", num_layers)
        
        
        
#         print("dataX[0].shape:", d[0].shape, len(d), "\n", 
#              "dataX_hat[0].shape:", dh[0].shape, len(dh)
#              )
#         PCA_Analysis (d, dh)
#         tSNE_Analysis (d, dh)
        visualization(d, dh, 'pca')
        visualization(d, dh, 'tsne')
        
        
        

In [ ]:
def plot_the_data(org_data, gen_data, low=0, up=100):
    n = org_data.shape[1]
    # fig, axs = plt.subplots(n, 1, figsize=(n, 1))
    for i in range(n):
        name = pld_complete_range_reduced.columns[i]
        plt.figure(figsize=(15, 8))
        plt.plot(org_data[low:up, i], 'g-', alpha=.9) 
        plt.plot(gen_data[low:up, i], 'b--', alpha=.7) 
        plt.legend(["Org", "TGAN"])
        plt.title(name)
        plt.show()
        plt.close()

    return None




In [ ]:
for seq_len in sequence_lens:
    for num_layers in number_layers:
        
        data_to_train = pld_complete_range_reduced.values
        n_samples = data_to_train.shape[0]
        n_time_series = data_to_train.shape[1]
        

        #%% Newtork Parameters
        parameters = dict()

        # parameters['hidden_dim'] = len(dataX[0][0,:]) * 4
        parameters['num_layers'] = num_layers
        parameters['iterations'] = iterations
        parameters['batch_size'] = 128
        parameters['module_name'] = 'gru'   # Other options: 'lstm' or 'lstmLN'
        # parameters['z_dim'] = len(dataX[0][0,:]) 

        
        d, dh, org, gen = load_results(seq_len=seq_len, 
                                       parameters=parameters, 
                                       name=name)
        
        print("seq_len:", seq_len, 
              "num_layers:", num_layers)


        plot_the_data(org_data=org, 
              gen_data=gen, 
              low=4000, up=4100)
        
        

In [ ]:
# seq_len = 3
# data_to_train = pld_complete_range_reduced.values
# n_samples = data_to_train.shape[0]
# n_time_series = data_to_train.shape[1]

# dataX = prepare_data_for_tgan(x=data_to_train, seq_length=seq_len)
# dataX[0].shape



In [ ]:
#%% Newtork Parameters
# parameters = dict()

# parameters['hidden_dim'] = len(dataX[0][0,:]) * 4
# parameters['num_layers'] = 3
# parameters['iterations'] = 10000
# parameters['batch_size'] = 128
# parameters['module_name'] = 'gru'   # Other options: 'lstm' or 'lstmLN'
# parameters['z_dim'] = len(dataX[0][0,:]) 

# print('Parameters are ' + str(parameters))



In [ ]:
# dataX_hat = tgan(dataX, parameters)   

# print('Finish Synthetic Data Generation')



In [ ]:
len(dataX_hat), dataX_hat[0].shape


In [ ]:
plot_the_data(org_data=org_data_gan_1_5,
              gen_data=gen_data_gan_1_5,
             low=9000, up=9300)



In [ ]:
plot_the_data(org_data=org_data_gan_10_3, 
              gen_data=gen_data_gan_10_3,
              low=9000, up=9300
             )



In [ ]:
plot_the_data(org_data=org_data_gan_10_5, 
              gen_data=gen_data_gan_10_5,
             low=9000, up=9300)



In [ ]:
plot_the_data(org_data=org_data_gan_50_3, 
              gen_data=gen_data_gan_50_3,
             low=9000, up=9300)





In [ ]:
# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE

In [ ]:
# def compute_pca(x_to_pca, n_components=2):
#     return PCA(n_components=n_components).fit_transform(x_to_pca)


# def compute_tsne(x_to_tsne, n_components=2):
#     return TSNE(n_components=n_components, n_iter=300).fit_transform(x_to_tsne)

In [ ]:
# x_org_pca = compute_pca(org_data_arr)
# x_tgan_pca = compute_pca(gen_data_arr)

In [ ]:

# plt.figure(figsize=(15, 8))
# plt.scatter(x_org_pca[:, 0], x_org_pca[:, 1], alpha=.9, c="g")
# plt.scatter(x_tgan_pca[:, 0], x_tgan_pca[:, 1], alpha=.2, c="b")

# plt.legend(["Org", "TGAN"])
# plt.show()

In [ ]:
# x_org_tsne = compute_tsne(org_data_arr)
# x_tgan_tsne = compute_tsne(gen_data_arr)

In [ ]:

# plt.figure(figsize=(15, 8))
# plt.scatter(x_org_tsne[:, 0], x_org_tsne[:, 1], alpha=.9, c="g")
# plt.scatter(x_tgan_tsne[:, 0], x_tgan_tsne[:, 1], alpha=.2, c="b")

# plt.legend(["Org", "TGAN"])
# plt.show()